<a href="https://colab.research.google.com/github/PedroPertusi/afd-monitor-jogadores/blob/main/data_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitor de notícias de jogadores de futebol - Campeonato Brasileiro Série A 2025

## Objetivo do Notebook
Este notebook tem como objetivo extrair e analisar notícias de jogadores de futebol do Campeonato Brasileiro Série A 2025, coletando dados do site Globo Esporte e organizando-os em um formato estruturado para análise posterior.

## Bibliotecas e módulos
- **BeautifulSoup**: extração de conteúdo HTML
- **requests**: requisições HTTP
- **re**: expressões regulares para limpeza de texto
- **pandas**: manipulação e análise de dados

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

Função para obter o HTML da página a ser acessada para extração dos dados

In [ ]:
def baixar_pagina(url):
    """
    Faz uma requisição HTTP GET para a URL fornecida e retorna o conteúdo HTML da página.

    Parâmetros:
    url (str): Endereço da página a ser baixada.

    Retorna:
    str: Conteúdo HTML da página como texto decodificado em UTF-8.
    """
    response = requests.get(url)
    response.encoding = "utf-8"
    return response.text

Função responsável por obter o link de uma notícia contido em seu respectivo título

In [ ]:
def extrair_links_noticias_post(url):
    """
    Extrai os links de notícias de um post de página HTML.

    Parâmetros:
    url (str): URL da página a partir da qual serão extraídos os links.

    Retorna:
    List[str]: Lista de URLs obtidas dos elementos <h2><a> encontrados na página.
    """
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    titulos = soup.find_all('h2')
    links_noticias = []
    for titulo in titulos:
        a_tag = titulo.find('a')
        if a_tag and a_tag.get('href'):
            links_noticias.append(a_tag.get('href'))
    return links_noticias


Função para extrair o conteúdo de cada notícia

In [ ]:
def extrair_conteudo_noticia(url):
    """
    Extrai o conteúdo textual de uma notícia a partir da URL fornecida,
    filtrando parágrafos relevantes e descartando intertítulos e listas.

    Parâmetros:
    url (str): Endereço da página da notícia a ser processada.

    Retorna:
    List[str]: Lista de strings contendo o texto limpo de cada parágrafo
               (<p> com classe 'content-text__container') sem intertítulos
               (div com classe 'content-intertitle') nem listas (<ul>).
    """
    html = baixar_pagina(url)
    soup = BeautifulSoup(html, 'html.parser')
    ps = soup.find_all('p', class_='content-text__container')

    textos = []
    for p in ps:
        # descarta parágrafos que contenham intertítulo ou lista
        if p.find('div', class_='content-intertitle') or p.find('ul'):
            continue
        # extrai texto limpo (inclui <a>, <strong>, etc)
        textos.append(p.get_text(" ", strip=True))
    return textos

Após a definição das funções necessárias para a extração de dados, podemos aplicá-las para realizar a extração

O site escolhido para a obtenção de notícias foi o Globo Esporte (https://ge.globo.com/)

In [5]:
times = ["atletico-mg", "bahia", "botafogo", "bragantino", "ceara", "corinthians", "cruzeiro", "flamengo", "fluminense", "fortaleza",
         "gremio", "internacional", "juventude", "mirassol", "palmeiras", "santos", "sao-paulo", "sport", "vasco", "vitoria"]

conteudos_times = {}

In [6]:
base_url = "https://ge.globo.com/futebol/times/"

for t in times:
  time_url = base_url + t
  noticias = extrair_links_noticias_post(time_url)

  conteudos = []
  for n in noticias:
    conteudos.append(extrair_conteudo_noticia(n))

  pattern = re.compile(r'^\+ O ge')
  c = [" ".join(parts) for parts in conteudos if not pattern.match(" ".join(parts))]

  c = [i for i in c if i not in (" ", "")]

  conteudos_times[t] = c

In [7]:
records = [
    (time, noticia)
    for time, noticias in conteudos_times.items()
    for noticia in noticias
]

df_time_noticia = pd.DataFrame(records, columns=['time', 'noticia'])

Para verificarmos se a extração foi feita corretamente, vamos exibir o DataFrame construído com os times e suas respectivas notícias.

In [8]:
df_time_noticia

,time,noticia
0,atletico-mg,Atlético-MG e Cienciano entram em campo nesta ...
1,atletico-mg,O meia-atacante Bernard foi a grande novidade ...
2,atletico-mg,"Antes de uma partida internacional, é normal o..."
3,atletico-mg,O carimbo de “atleta da base” assombrou Rubens...
4,bahia,"O Bahia perdeu de virada, por 2 a 1 com o Inte..."
...,...,...
89,vitoria,O Vitória sofreu mais um duro golpe na tempora...
90,vitoria,O Vitória foi eliminado da Copa Sul-Americana ...
91,vitoria,O Vitória alcançou o seu principal objetivo pa...
92,vitoria,Agora você pode acompanhar a cobertura do ge t...


Após a validação do DataFrame, podemos salvar estas informações dentro de um arquivo .csv

In [9]:
df_time_noticia.to_csv("times_noticias.csv")

## Conclusão
Neste notebook realizamos a extração de notícias de todos os times da Série A 2025, validamos o DataFrame resultante e salvamos os dados em um arquivo CSV para futuras análises.